# Кто не умеет - тот учит)

Видимо эта фраза все же мудра, потому что для меня лично, лучший способ понять что-то - это объяснить или рассказать кому-то другому. Поэтому если так случится, что кто-то будет это читать, то знайте, я прямо сейчас или в недалеком прошлом сижу и разбираюсь с подробностями того, как утроена жизнь в python. На этот раз в той его части, которая отвечает за пыполнение многопоточных программ. 

# Пример 1. Однопоточное выполение

Кажется, что большинство существующего кода в мире именно так и работает. Последовательно выполняет заданную серию команд. Давайте сначала разберемся "на кошках". На элементарных примерах

In [10]:
from time import ctime, sleep


def loop0():
    print 'start loop 0 at:', ctime()
    sleep(4)
    print 'loop 0 done at:', ctime()

    
def loop1():
    print 'start loop 1 at:', ctime()
    sleep(2)
    print 'loop 1 done at:', ctime()
    
    
def main():
    print 'starting at:', ctime()
    loop0()
    loop1()
    print 'all DONE at:', ctime()
    
main()

starting at: Sat Dec 12 17:41:42 2015
start loop 0 at: Sat Dec 12 17:41:42 2015
loop 0 done at: Sat Dec 12 17:41:46 2015
start loop 1 at: Sat Dec 12 17:41:46 2015
loop 1 done at: Sat Dec 12 17:41:48 2015
all DONE at: Sat Dec 12 17:41:48 2015


Понятно, что в случае, когда loop0 и loop1 делаю что-то независимое, нет нужды выполнять их последовательно, если это можно сделать одновременно. Поэтому другой пример

In [11]:
import thread


def main():
    print 'starting at:', ctime()
    thread.start_new_thread(loop0, ())
    thread.start_new_thread(loop1, ())
    sleep(6)
    print 'all DONE at:', ctime()

In [12]:
main()

starting at: Sat Dec 12 17:41:48 2015
start loop 0 at: Sat Dec 12 17:41:48 2015
start loop 1 at: Sat Dec 12 17:41:48 2015
loop 1 done at: Sat Dec 12 17:41:50 2015
loop 0 done at: Sat Dec 12 17:41:52 2015
all DONE at: Sat Dec 12 17:41:54 2015


Видно, что тут функции выполнились параллельно. Стартовали примерно в одно и то же время, закончились за 4 и 2 секунды соответственно. Но в основной программе мы зачем-то написали sleep(6). Зачем? А затем, что если этого не сделать, то основной процесс завершится сразу, как только запустит функции loop0 и loop1. А как только завершается основной процесс, завершаются еще и все порожденные процессы. А в модуле thread еще к тому же не проиходит очистка памяти от таких принудительно завершенных потоков. А в модуле threading происходит. Об этом подробнее будет далее, но прямо сейчас можно сказать следущее. Модуль thread здесь используется только для ознакомительных целей. А вообще пользоваться им нужно, только если вы в нем хорош разбираетесь и уверены, что сможете с его помощью написать хорошую надежную программу. Если же нет и вам просто нужно что-то многопоточное на python - используйте модуль threading. Он является более высокоуровневым. Примеры с использованием модуля theading будут далее. 

In [15]:
def main():
    print 'starting at:', ctime()
    thread.start_new_thread(loop0, ())
    thread.start_new_thread(loop1, ())
    print 'all DONE at:', ctime()
    
main()

starting at: Sat Dec 12 18:04:09 2015
all DONE at: Sat Dec 12 18:04:09 2015
start loop 0 at: Sat Dec 12 18:04:09 2015
start loop 1 at: Sat Dec 12 18:04:09 2015
loop 1 done at: Sat Dec 12 18:04:11 2015


Тут наглядно видно, что произлошло, когда мы не подождали завершения дочерных процессов, а сразу вышли из родительского процесса. Один дочерний завершился, а второй - нет. Потому что нет гарантий такого завершения при использовании модуля thread.

Чтобы иметь какие-то гарантии о том, что родительский процесс дождется окончания дочерних потоков, используют блокировки. Рассмотрим пример.

In [19]:
loops = [2, 4]

def loop(index, n_seconds, lock):
    print 'start loop', index, ' at:' , ctime()
    sleep(n_seconds)
    print 'loop', index, ' done at:', ctime()
    lock.release()
    
    
def main():
    print 'starting at:', ctime()
    locks = []
    n_loops = xrange(len(loops))
    
    for i in n_loops:
        lock = thread.allocate_lock()   # заводим объект блокировки
        lock.acquire()                  # захватываем этот объект, то есть блокируем доступ
        locks.append(lock)              # и запоминаем все такие объекты в лист locks
        
    for i in n_loops:                   # в новом треде запускаем функцию loop с параметрами
                                        # индекс, число секунд, которое надо спать
                                        # объект блокировки, чтобы после окончания работы функции его разблокировать
        thread.start_new_thread(loop, (i, loops[i], locks[i]))
        
    for i in n_loops:                   # этот код просто ждет, пока все объекты блокировки в массиве locks
                                        # разблокируются. 
        while locks[i].locked(): 
            pass
    print 'all DONE at:', ctime()
    
    
main()

starting at: Sat Dec 12 18:19:00 2015
start loop 1  at: Sat Dec 12 18:19:00 2015
start loop 0  at: Sat Dec 12 18:19:00 2015
loop 0  done at: Sat Dec 12 18:19:02 2015
loop 1  done at: Sat Dec 12 18:19:04 2015
all DONE at: Sat Dec 12 18:19:04 2015


# Модуль threading

Сначала еще про недостаток модуля thread. В модуле thread не поддерживается принцип организации работы потоков на основе демонов. Это когда можно завершить родительский процесс, дочерние продолжат работу. Вот с помощью модул thread так сделать не получится. А помощью модуля threading - можно. Примеры будут ниже. 

Про класс threading.Thread

Можно посмотреть на "публичные" методы и атрибуты класса (в явном виде понятия публичных метдов в python нет, но методы, которые начинаются на _, считаются приватными).

In [28]:
import threading
filter( lambda line: not line.startswith('_'), dir(threading.Thread))

['daemon',
 'getName',
 'ident',
 'isAlive',
 'isDaemon',
 'is_alive',
 'join',
 'name',
 'run',
 'setDaemon',
 'setName',
 'start']